In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import joblib

# Load dataset
csv_path = "/content/drive/MyDrive/fsdbus/stMTMV_water_quality_dataset.csv"
df = pd.read_csv(csv_path)

# Target encoding
df['Water_Quality_Label'] = df['Water_Quality_Label'].map({'Safe': 0, 'Unsafe': 1})

# Define selected features
selected_features = [
    "Temperature", "pH", "Chlorine_Level",
    "Pipe_Age", "Pipe_Length", "Pipe_Diameter",
    "Turbidity", "Conductivity", "Water_Pressure"
]

# Define feature matrix X and target vector y
X = df[selected_features]
y = df['Water_Quality_Label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(9,)),  # Changed from 10 to 9 (number of features)
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Save model and scaler
model.save("/content/drive/MyDrive/rcdc/RCDecay_model.h5")
joblib.dump(scaler, "/content/drive/MyDrive/rcdc/RCDecay_scaler.save")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/fsdbus/stMTMV_water_quality_dataset.csv'

In [ ]:
# Imports
import numpy as np
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
from tensorflow.keras.models import load_model
import tensorflow as tf

# Load trained model and scaler
model = load_model("/content/drive/MyDrive/rcdc/RCDecay_model.h5", compile=False)
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanAbsoluteError()])
scaler = joblib.load("/content/drive/MyDrive/rcdc/RCDecay_scaler.save")

# Feature names (match model training)
feature_names = [
    "Temperature", "pH", "Chlorine_Level",
    "Pipe_Age", "Pipe_Length", "Pipe_Diameter",
    "Turbidity", "Conductivity", "Water_Pressure"
]

# Create input widgets
input_widgets = {
    name: widgets.FloatText(
        description=name,
        layout=widgets.Layout(width='400px'),
        style={'description_width': '250px'}
    ) for name in feature_names
}

# Prediction button and output box
submit_button = widgets.Button(description="Predict RC Decay", button_style='success')
output = widgets.Output()

# Prediction logic
def on_predict_clicked(b):
    with output:
        clear_output()
        try:
            # Collect input values
            values = [input_widgets[name].value for name in feature_names]
            sample = np.array([values])
            sample_scaled = scaler.transform(sample)

            # Predict
            pred_rc = model.predict(sample_scaled, verbose=0)
            rc_level = round(pred_rc[0][0], 3)

            is_safe = rc_level >= 0.2  # Safety threshold

            print("🔍 Prediction Results")
            print(f"☣️  Residual Chlorine Level: {rc_level} mg/L")
            print("💧 Safety Status:", "✅ Safe" if is_safe else "❌ Unsafe")

            # Feature importance
            input_tensor = tf.convert_to_tensor(sample_scaled, dtype=tf.float32)
            with tf.GradientTape() as tape:
                tape.watch(input_tensor)
                prediction = model(input_tensor)
            grads = tape.gradient(prediction, input_tensor).numpy()[0]
            importance = np.abs(grads)
            ranked_indices = np.argsort(-importance)

            print("\n📊 Ranked Feature Importance:")
            for rank, idx in enumerate(ranked_indices):
                fname = feature_names[idx]
                val = round(values[idx], 3)
                imp_score = round(importance[idx], 5)
                print(f"{rank+1}. {fname} (Input: {val}) → Contribution: {imp_score}")

            top_feature = feature_names[ranked_indices[0]]
            top_score = round(importance[ranked_indices[0]], 5)
            print(f"\n🧠 Most Influential Factor: '{top_feature}' (Score: {top_score})")

        except Exception as e:
            print("❌ Error:", str(e))

# Bind and display UI
submit_button.on_click(on_predict_clicked)
display(widgets.VBox(list(input_widgets.values()) + [submit_button, output]))
